<a href="https://colab.research.google.com/github/Sayandip2023/Text-To-SQL/blob/main/Text_To_SQL_Using_LLM_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the necessary libraries

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import sqlite3
import sqlglot
import re

## Loading the model and the tokenizer

In [2]:
# Load the pre-trained model and tokenizer
model_path = 'gaussalgo/T5-LM-Large-text2sql-spider'
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/813 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

## Setting up the database

In [3]:
# Setup in-memory SQLite database
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create sample tables and insert data
cursor.execute("CREATE TABLE Employees (id INTEGER, name TEXT, age INTEGER, salary REAL, department_id INTEGER)")
cursor.execute("INSERT INTO Employees VALUES (1, 'Alice', 30, 60000.0, 1)")
cursor.execute("INSERT INTO Employees VALUES (2, 'Bob', 35, 70000.0, 1)")
cursor.execute("INSERT INTO Employees VALUES (3, 'Charlie', 28, 50000.0, 2)")
cursor.execute("INSERT INTO Employees VALUES (4, 'David', 40, 80000.0, 3)")
cursor.execute("INSERT INTO Employees VALUES (5, 'Eve', 45, 90000.0, 4)")

cursor.execute("CREATE TABLE Departments (id INTEGER, department_name TEXT)")
cursor.execute("INSERT INTO Departments VALUES (1, 'Engineering')")
cursor.execute("INSERT INTO Departments VALUES (2, 'Marketing')")
cursor.execute("INSERT INTO Departments VALUES (3, 'Sales')")
cursor.execute("INSERT INTO Departments VALUES (4, 'HR')")

cursor.execute("CREATE TABLE Projects (id INTEGER, project_name TEXT, budget REAL, department_id INTEGER)")
cursor.execute("INSERT INTO Projects VALUES (1, 'Project Apollo', 200000.0, 1)")
cursor.execute("INSERT INTO Projects VALUES (2, 'Project Zeus', 300000.0, 2)")
cursor.execute("INSERT INTO Projects VALUES (3, 'Project Hera', 100000.0, 3)")
cursor.execute("INSERT INTO Projects VALUES (4, 'Project Poseidon', 100000.0, 4)")

conn.commit()

## Designing the schema

In [4]:
# Defining schema
schema = """
   "Employees" "id" INTEGER, "name" TEXT, "age" INTEGER, "salary" REAL, "department_id" INTEGER, primary key: "id" [SEP]
   "Departments" "id" INTEGER, "department_name" TEXT, primary key: "id" [SEP]
   "Projects" "id" INTEGER, "project_name" TEXT, "budget" REAL, "department_id" INTEGER, primary key: "id"
"""

## Defining Ground Truth

In [5]:
# Defining natural language queries and their expected results
queries = [
    {
        "nl_query": "Find the total number of employees.",
        "expected_sql": "SELECT COUNT(*) FROM Employees",
        "expected_result": [(5,)]
    },
    {
        "nl_query": "What is the average salary of employees?",
        "expected_sql": "SELECT AVG(salary) FROM Employees",
        "expected_result": [(70000.0,)]
    },
    {
        "nl_query": "List the names of employees who have a salary greater than 60000.",
        "expected_sql": "SELECT name FROM Employees WHERE salary > 60000",
        "expected_result": [('Bob',), ('David',), ('Eve',)]
    },
    {
        "nl_query": "What is the total budget of all projects?",
        "expected_sql": "SELECT SUM(budget) FROM Projects",
        "expected_result": [(700000.0,)]
    },
    {
        "nl_query": "List the names of employees who have a salary greater than 80000.",
        "expected_sql": "SELECT name FROM Employees WHERE salary > 80000",
        "expected_result": [('Eve',)]
    },
    {
        "nl_query": "List all departments.",
        "expected_sql": "SELECT department_name FROM Departments",
        "expected_result": [('Engineering',), ('Marketing',), ('Sales',), ('HR',)]
    },
    {
        "nl_query": "Find the highest salary among employees.",
        "expected_sql": "SELECT MAX(salary) FROM Employees",
        "expected_result": [(90000.0,)]
    },
    {
        "nl_query": "Find the average budget of projects.",
        "expected_sql": "SELECT AVG(budget) FROM Projects",
        "expected_result": [(175000.0,)]
    },
    {
        "nl_query": "List the names and ages of employees who are younger than 35.",
        "expected_sql": "SELECT name, age FROM Employees WHERE age < 35",
        "expected_result": [('Alice', 30), ('Charlie', 28)]
    },
    {
        "nl_query": "Find the total budget of projects for the HR department.",
        "expected_sql": "SELECT SUM(budget) FROM Projects WHERE department_id = (SELECT id FROM Departments WHERE department_name = 'HR')",
        "expected_result": [(100000.0,)]
    },
    {
        "nl_query": "List the project names with a budget between 100000 and 200000.",
        "expected_sql": "SELECT project_name FROM Projects WHERE budget BETWEEN 100000 AND 200000",
        "expected_result": [('Project Apollo',), ('Project Hera',), ('Project Poseidon',)]
    }
]
queries.extend([
    # More Queries
    {
        "nl_query": "List the names of employees who are exactly 30 years old.",
        "expected_sql": "SELECT name FROM Employees WHERE age = 30",
        "expected_result": [('Alice',)]
    },
    {
        "nl_query": "What is the sum of the salaries of all employees in the Marketing department?",
        "expected_sql": "SELECT SUM(salary) FROM Employees WHERE department_id = (SELECT id FROM Departments WHERE department_name = 'Marketing')",
        "expected_result": [(50000.0,)]
    },
    {
        "nl_query": "Find the average age of employees.",
        "expected_sql": "SELECT AVG(age) FROM Employees",
        "expected_result": [(35.6,)]
    },
    {
        "nl_query": "List all projects with a budget greater than 150000.",
        "expected_sql": "SELECT project_name FROM Projects WHERE budget > 150000",
        "expected_result": [('Project Apollo',), ('Project Zeus',)]
    },
    {
        "nl_query": "Find the name of the employee with the lowest salary.",
        "expected_sql": "SELECT name FROM Employees WHERE salary = (SELECT MIN(salary) FROM Employees)",
        "expected_result": [('Charlie',)]
    },
    {
        "nl_query": "Find the department with the highest number of employees.",
        "expected_sql": "SELECT department_name FROM Departments WHERE id = (SELECT department_id FROM Employees GROUP BY department_id ORDER BY COUNT(*) DESC LIMIT 1)",
        "expected_result": [('Engineering',)]
    },
    {
        "nl_query": "List the names of employees who have a salary between 60000 and 80000.",
        "expected_sql": "SELECT name FROM Employees WHERE salary BETWEEN 60000 AND 80000",
        "expected_result": [('Alice',), ('Bob',), ('David',)]
    },
    {
        "nl_query": "Find the total number of projects.",
        "expected_sql": "SELECT COUNT(*) FROM Projects",
        "expected_result": [(4,)]
    },
    {
        "nl_query": "Find the total salary of employees in the Sales department.",
        "expected_sql": "SELECT SUM(salary) FROM Employees WHERE department_id = (SELECT id FROM Departments WHERE department_name = 'Sales')",
        "expected_result": [(80000.0,)]
    },
    {
        "nl_query": "Find the employee with the highest age.",
        "expected_sql": "SELECT name FROM Employees WHERE age = (SELECT MAX(age) FROM Employees)",
        "expected_result": [('Eve',)]
    },
    {
        "nl_query": "List the names of employees whose names start with 'A'.",
        "expected_sql": "SELECT name FROM Employees WHERE name LIKE 'A%'",
        "expected_result": [('Alice',)]
    },
    {
        "nl_query": "What is the total salary of all employees who are younger than 40?",
        "expected_sql": "SELECT SUM(salary) FROM Employees WHERE age < 40",
        "expected_result": [(180000.0,)]
    },
    {
        "nl_query": "Find the names of employees who have the same salary as 'Bob'.",
        "expected_sql": "SELECT name FROM Employees WHERE salary = (SELECT salary FROM Employees WHERE name = 'Bob')",
        "expected_result": [('Bob',)]
    },
    {
        "nl_query": "List the names of employees who have worked on projects with a budget over 150000.",
        "expected_sql": "SELECT DISTINCT E.name FROM Employees E JOIN Projects P ON E.department_id = P.department_id WHERE P.budget > 150000",
        "expected_result": [('Alice',), ('Bob',)]
    },
    {
        "nl_query": "Find the number of projects with a budget greater than 100000.",
        "expected_sql": "SELECT COUNT(*) FROM Projects WHERE budget < 100000",
        "expected_result": [(2,)]
    },
    {
        "nl_query": "Find the average salary of employees in the Marketing department.",
        "expected_sql": "SELECT AVG(salary) FROM Employees WHERE department_id = (SELECT id FROM Departments WHERE department_name = 'Marketing')",
        "expected_result": [(50000.0,)]
    },
    {
        "nl_query": "List the project names and their budgets where the budget is not equal to 100000.",
        "expected_sql": "SELECT project_name, budget FROM Projects WHERE budget <> 100000",
        "expected_result": [('Project Apollo', 200000.0), ('Project Zeus', 300000.0)]
    },
    {
        "nl_query": "Find the total number of employees in each department.",
        "expected_sql": "SELECT department_id, COUNT(*) FROM Employees GROUP BY department_id",
        "expected_result": [(1, 2), (2, 1), (3, 1), (4, 1)]
    },
    {
        "nl_query": "List the project names and their budgets in descending order of budget.",
        "expected_sql": "SELECT project_name, budget FROM Projects ORDER BY budget DESC",
        "expected_result": [('Project Zeus', 300000.0), ('Project Apollo', 200000.0), ('Project Hera', 100000.0), ('Project Poseidon', 100000.0)]
    },
    {
        "nl_query": "Find the total salary of employees who are older than 30 and younger than 50.",
        "expected_sql": "SELECT SUM(salary) FROM Employees WHERE age > 30 AND age < 50",
        "expected_result": [(240000.0,)]
    },
    {
        "nl_query": "List the names and salaries of employees who have the highest salary.",
        "expected_sql": "SELECT name, salary FROM Employees WHERE salary = (SELECT MAX(salary) FROM Employees)",
        "expected_result": [('Eve', 90000.0)]
    },
])

## Generating SQL queries

In [6]:
# Function to generate SQL query from natural language query
def generate_sql_query(nl_query, schema):
    input_text = f"Question: {nl_query} Schema: {schema}"
    model_inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**model_inputs, max_length=512)
    output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip()
    return output_text

## Correct misising characted in the generated query

In [7]:
# Function to correct common SQL syntax errors
def correct_sql_syntax(sql_query):
    # Basic correction for missing operators
    sql_query = re.sub(r'\b(age)\s*(\d+)', r'\1 < \2', sql_query)  # Correct missing '<' for age
    sql_query = re.sub(r'\b(salary)\s*(\d+)', r'\1 < \2', sql_query)  # Correct missing '<' for salary
    return sql_query

## Validating SQL syntax

In [8]:
# Function to validate SQL syntax
def validate_sql_syntax(sql_query):
    try:
        sqlglot.parse(sql_query)
        return True
    except Exception as e:
        return False

## Designing Output Section

In [9]:
correct_predictions = 0
total_queries = len(queries)

for idx, query_info in enumerate(queries, start=1):
    nl_query = query_info['nl_query']
    expected_sql = query_info['expected_sql']
    expected_result = query_info['expected_result']

    # Generate SQL using the model
    generated_sql = generate_sql_query(nl_query, schema)
    corrected_sql = correct_sql_syntax(generated_sql)

    # Validate SQL syntax
    is_valid_sql = validate_sql_syntax(corrected_sql)
    print(f"Natural Language Query {idx}: {nl_query}")
    print(f"Generated SQL: {generated_sql}")
    print(f"Corrected SQL: {corrected_sql}")
    print(f"Valid SQL: {'Yes' if is_valid_sql else 'No'}")
    print(f"Expected SQL: {expected_sql}")

    if not is_valid_sql:
        print("SQL is invalid, skipping execution.")
        print("-" * 80)
        continue

    try:
        # Ensure SQL is formatted correctly
        if not corrected_sql.lower().startswith("select"):
            print(f"Invalid SQL format: {corrected_sql}")
            continue

        # Execute corrected SQL
        cursor.execute(corrected_sql)
        generated_result = cursor.fetchall()
        print(f"Generated Result: {generated_result}")
        print(f"Expected Result: {expected_result}")

        # Check if the results match
        if generated_result == expected_result:
            correct_predictions += 1
            print("Matches Expected: True")
        else:
            print("Matches Expected: False")
    except Exception as e:
        print(f"Error executing corrected SQL: {e}")

    print("-" * 80)

# Calculate and print accuracy
accuracy = (correct_predictions / total_queries) * 100
print(f"Accuracy: {accuracy:.2f}%")

# Close connection
conn.close()

Natural Language Query 1: Find the total number of employees.
Generated SQL: SELECT count(*) FROM employees
Corrected SQL: SELECT count(*) FROM employees
Valid SQL: Yes
Expected SQL: SELECT COUNT(*) FROM Employees
Generated Result: [(5,)]
Expected Result: [(5,)]
Matches Expected: True
--------------------------------------------------------------------------------
Natural Language Query 2: What is the average salary of employees?
Generated SQL: SELECT avg(salary) FROM employees
Corrected SQL: SELECT avg(salary) FROM employees
Valid SQL: Yes
Expected SQL: SELECT AVG(salary) FROM Employees
Generated Result: [(70000.0,)]
Expected Result: [(70000.0,)]
Matches Expected: True
--------------------------------------------------------------------------------
Natural Language Query 3: List the names of employees who have a salary greater than 60000.
Generated SQL: SELECT name FROM employees WHERE salary > 60000
Corrected SQL: SELECT name FROM employees WHERE salary > 60000
Valid SQL: Yes
Expecte